In [1]:
import pandas as pd
import numpy as np

# 1. 파일 경로 설정
DATA_PATH = '../../data/raw/'
SUB_PATH = './submissions/'

# 2. 결과 데이터 로드
# Exp 26: 현재 가장 안정적인 Private 점수를 보유한 SOTA 모델
exp26 = pd.read_csv(f'{SUB_PATH}26_m1_m7_Rank_Ensemble_0.77364.csv')
# Exp 37: Pseudo-Labeling을 통해 테스트셋의 패턴을 반영한 모델
exp37 = pd.read_csv(f'{SUB_PATH}37_Pseudo_Labeling_m7_AUC_0.79405.csv')

# 3. Rank Averaging 수행
# 각 모델의 예측값(voted)을 순위(Rank)로 변환 (0~1 사이로 정규화)
rank26 = exp26['voted'].rank(pct=True)
rank37 = exp37['voted'].rank(pct=True)

# 4. 7:3 가중치 적용 결합
# Exp 26의 안정적인 일반화 성능에 70%, Exp 37의 새로운 시그널에 30% 배분
final_rank = (rank26 * 0.7) + (rank37 * 0.3)

# 5. 최종 제출 파일 생성
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')

# Exp 26의 기존 점수 분포를 유지하면서 순위만 새로운 결합 순위로 재배치
# 이는 모델의 출력 스케일을 안정적으로 유지하기 위함입니다
submission['voted'] = exp26['voted'].sort_values().values[
    (final_rank * (len(final_rank) - 1)).astype(int)
]

# 결과 저장
output_file = f'{SUB_PATH}38_Ensemble_73.csv'
submission.to_csv(output_file, index=False)

print(f'Exp 38: Final Master Ensemble (7:3) file created at {output_file}')

Exp 38: Final Master Ensemble (7:3) file created at ./submissions/38_Ensemble_73.csv


In [2]:
import pandas as pd
import numpy as np

# 1. 경로 및 파일 설정
SUB_PATH = './submissions/'
DATA_PATH = '../../data/raw/'

# Exp 26: 안정적인 성능의 기존 SOTA (0.78276 / 0.78019)
exp26 = pd.read_csv(f'{SUB_PATH}26_m1_m7_Rank_Ensemble_0.77364.csv')
# Exp 37: 테스트셋 패턴을 학습한 PL 모델 (0.78198 / 0.77958)
exp37 = pd.read_csv(f'{SUB_PATH}37_Pseudo_Labeling_m7_AUC_0.79405.csv')

# 2. 순위 기반 표준화 (Rank Averaging)
# 두 모델의 예측값(voted)을 순위로 변환하여 스케일 차이 제거
rank26 = exp26['voted'].rank(pct=True)
rank37 = exp37['voted'].rank(pct=True)

# 3. 5:5 균등 가중치 적용 (Exp 26의 일반화 로직 계승)
# 모델 간의 균형을 맞추어 특정 모델의 과적합 노이즈를 상쇄
final_rank = (rank26 * 0.5) + (rank37 * 0.5)

# 4. 최종 결과 생성 및 저장
submission = pd.read_csv(f'{DATA_PATH}sample_submission.csv')

# Exp 26의 안정적인 값 분포(1.1~1.9)를 유지하며 순위만 재배치
submission['voted'] = exp26['voted'].sort_values().values[
    (final_rank * (len(final_rank) - 1)).astype(int)
]

output_name = f'{SUB_PATH}38_Master_Ensemble_55.csv'
submission.to_csv(output_name, index=False)

print(f'Exp 38: 5:5 Rank Ensemble created -> {output_name}')

Exp 38: 5:5 Rank Ensemble created -> ./submissions/38_Master_Ensemble_55.csv
